# [실습] 오픈소스 LLM을 활용한 챗봇 만들기


## 실습 목표
---
- LangChain을 활용해서 Mistral 7B 모델을 사용하는 챗봇을 개발합니다.
- 짧은 Chain을 구성하고, 이를 활용해서 챗봇을 구현합니다.

## 실습 목차
---

1. **ChatOllama Agent 생성:** 사용자의 입력에 대한 Mistral 7B 모델의 답변을 받아오는 Agent를 생성합니다.

2. **챗봇 Chain 구성**: ChatOllama Agent를 비롯하여 챗봇 구현에 필요한 Agent들을 엮어서 챗봇 Chain으로 구성합니다.

3. **챗봇 사용**: 여러분이 구성하신 챗봇을 사용해봅니다.

## 실습 개요
---

LangChain의 Chain을 활용해서 Mistral 7B 모델을 활용하는 챗봇을 구현하고, Chain을 형성하는 방법을 이해합니다.


## 0. 환경 설정
- 필요한 라이브러리를 불러옵니다.

In [1]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama

- Ollama를 통해 Mistral 7B 모델을 불러옵니다.

In [2]:
# 이 실습은 nohup ollama serve & 명령어를 통해 백그라운드에서 ollama serve 명령을 실행한 상태입니다.
# 여러분이 현업에서 사용할 때에는 위 명령어를 통해 serve를 백그라운드에서 실행하는 것이 좋습니다.
!ollama pull mistral:7b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB           

## 1. ChatOllama Agent 생성


Mistral 7B 모델을 사용하는 ChatOllama Agent를 생성합니다. 
- ChatOllama Agent는 사용자의 입력을 Ollama를 통해 로컬에서 구동한 LLM에 전송하고, 그 답변을 반환합니다.
- 본 RAG 과정에서는 LLM으로 ChatOllama와 오픈 소스 LLM을 활용할 것입니다.

In [3]:
# 먼저, mistral:7b 모델을 사용하는 ChatOllama 객체를 생성합니다.
llm = ChatOllama(model="mistral:7b")

Agent를 구성했으니, 이제 Agent를 사용해봅시다.

### 1-1. Runnable interface

LangChain에서 Chain으로 엮을 수 있는 대부분의 구성 요소 (Agent, Tool 등..)는 "Runnable" protocol을 공유합니다.
- 관련 LangChain API 문서: [langchain_core.runnables.base.Runnable — 🦜🔗 LangChain 0.1.4](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable)

Runnable protocol을 공유하는 구성 요소는 모두 아래 세 메서드를 가지고 있습니다:
- stream: 구성 요소의 답변을 순차적으로 반환한다 (stream back)
- invoke: 입력된 값으로 chain을 호출하고, 그 결과를 반환한다.
- batch: 입력값 리스트 (batch)로 chain을 호출하고, 그 결과를 반환한다.

예시로, 저희가 방금 사용한 `ChatOllama` Class는 "Runnable" 하기 때문에 `invoke` 메서드를 가지고 있습니다.
- invoke() 메서드를 통해 Agent, Chain 등에 데이터를 입력하고, 그 출력을 받아올 수 있습니다.

`invoke` 메서드를 사용해봅시다. 여기서는 "당신은 누구입니까?" 라는 질문을 입력하면 Agent가 OpenAI API를 통해 Mistral 7B 모델의 답변을 받아 출력할 것입니다.

In [4]:
llm.invoke("당신은 누구입니까?")

AIMessage(content=' 제가 프로그램으로서의 존재로서 인간이 아닙니다. 저는 디지털 대화 시스템으로, Microsoft의 Azure Bot Service를 통해 구현되었습니다.\n\n나는 AI(인공지능)를 사용하여 문장에서 의도와 내용을 파악하고, 이를 기반으로 적절한 답변을 제공합니다. 항상 친절하고 유리한 대화를 보장할 수는 없지만, 최선을 다하겠습니다.', response_metadata={'model': 'mistral:7b', 'created_at': '2024-08-08T04:59:08.940118378Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 5806650304, 'load_duration': 1488070839, 'prompt_eval_count': 15, 'prompt_eval_duration': 63674000, 'eval_count': 174, 'eval_duration': 4253211000}, id='run-18555c4d-072e-4781-b405-02ee3c6d63b0-0')

단순 텍스트 뿐만 아니라, 시스템, 사람, AI의 답변을 리스트로 정리하여 입력할 수 있습니다. 

여기서는 LangChain의 `SystemMessage`, `HumanMessage` Class를 활용해봅시다.

In [5]:
messages = [
    SystemMessage("당신은 친절한 AI 어시스턴트 입니다."),
    HumanMessage("당신을 소개해주세요."),
]

response = llm.invoke(messages)

시스템 프롬프트에 '친절한 AI 어시스턴트' 라는 역할을 명시하였습니다.

이제 Mistral 7B 모델이 아까와 같은 질문에 어떻게 답했는지 확인해봅시다.

In [6]:
response

AIMessage(content=' 안녕하세요! 제가 WanderAI라고 합니다. AI 어시스턴트로써, 여러분의 질문과 문제들에 대한 도움이 되기를 바랍니다. 나는 다양한 영역에서 지식을 가지고 있습니다. 예를 들어, 전자학습, 프로그래밍, 기술, 생활 등 여러분이 궁금한 점들에 대해 도움이 되는 정보와 방법을 소개하고 설명하려니마땅다듭니다. 또, 나는 기본적으로 친절하고 신뢰성 있는 서비스를 제공합니다. 여러분이 어떤 도움이 필요한지에 대해 궁금하시면, "WanderAI, 어떤 도움이 되는 일을 할 수 있나요?"라고 말씀해주세요!', response_metadata={'model': 'mistral:7b', 'created_at': '2024-08-08T05:03:21.657007479Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 7762577900, 'load_duration': 18456432, 'prompt_eval_count': 40, 'prompt_eval_duration': 85407000, 'eval_count': 305, 'eval_duration': 7527187000}, id='run-cad9432f-6ea7-434a-a1cd-c6a4b6b8df2a-0')

같은 질문을 했음에도 자신을 소개하는 문구가 조금 달라진 것 을 확인할 수 있습니다.

### [TODO] 다양한 역할을 적용해서 어떻게 답변이 달라지는지 자유롭게 실험해보세요.

In [18]:
role = '수학선생님'
messages = [
    SystemMessage(f"당신은 {role} 입니다."),
    HumanMessage("당신을 소개해주세요."),
]

response = llm.invoke(messages)
response

AIMessage(content=' 안녕하세요! 제 이름은 AI로서 수학선생으로 작용하는 AI-Math-Teacher입니다. 사람과 달리 24시간 일기 않아도, 100%의 정확도로 수학 문제를 해결할 수 있습니다. 또한 문제의 난이도에 관계없이 도움이 될 수 있습니다. 어떤 수학 과목이든 간단하고 복잡한 내용까지 안내해드리고, 문제를 풀어주거나 설명하는 등 여러분의 수학 학습을 돕도록 노력하겠습니다.', response_metadata={'model': 'mistral:7b', 'created_at': '2024-08-08T05:09:10.777781803Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 5272279021, 'load_duration': 17233307, 'prompt_eval_count': 33, 'prompt_eval_duration': 25066000, 'eval_count': 208, 'eval_duration': 5100180000}, id='run-5465979e-4343-4eaa-aafa-6bac89cddc6e-0')

## 2. 챗봇 Chain 구성

조금 전 `llm` object의 반환 값을 확인해보면, 다른 챗봇을 쓸 때 처럼 답변만 출력된 것이 아니라 다양한 메타 데이터 까지 같이 출력된 것을 확인할 수 있습니다.

저희가 ChatGPT를 쓸 때를 생각해보면, 챗봇에 이걸 그대로 출력하는건 좀 부자연스럽습니다.

이를 방지하기 위해, 답변을 parsing하는 `StrOutputParser`를 활용해봅시다.

### 2-1. Output Parser
- ChatOllama Agent를 비롯하여 LLM 답변 중 content만 자동으로 추출하는 Tool인 `StrOutputParser`를 사용합니다.

In [13]:
parser = StrOutputParser()

`StrOutputParser`를 사용해봅시다.

In [19]:
# Parser가 제대로 답변만을 리턴하는지 확인합니다.
parsed_response = parser.invoke(response)
print(parsed_response)

 안녕하세요! 제 이름은 AI로서 수학선생으로 작용하는 AI-Math-Teacher입니다. 사람과 달리 24시간 일기 않아도, 100%의 정확도로 수학 문제를 해결할 수 있습니다. 또한 문제의 난이도에 관계없이 도움이 될 수 있습니다. 어떤 수학 과목이든 간단하고 복잡한 내용까지 안내해드리고, 문제를 풀어주거나 설명하는 등 여러분의 수학 학습을 돕도록 노력하겠습니다.


response에서 의도한 대로 텍스트만 추출하는 것을 확인할 수 있습니다.

### 2-2. 간단한 체인 구성

- 저희는 `ChatOllama` 를 통해 Mistral 7B 모델의 답변을 받았고, 그 받은 답변을 다시 `StrOutputParser`에 입력해서 답변만 추출하였습니다.
- 이 과정을 Chain으로 엮어서 간략화 해봅시다.

In [20]:
# pipe (|) 연산자를 통해 두 객체를 연결해서 하나의 체인으로 만들 수 있습니다.
chain = llm | parser

Chain 역시 "Runnable" 하므로, `invoke` 메서드를 통해 Chain의 각 구성요소의 `invoke` 메서드를 순차적으로 호출할 수 있습니다.

이때 특정 객체의 `invoke` 반환값은 Chain 상에서 연결된 다음 객체의 `invoke` 메서드에 입력됩니다.

In [21]:
# 체인을 실행하면, 체인에 포함된 모든 객체가 순차적으로 실행되며, 마지막 객체의 결과가 반환됩니다.
# 여기서는 llm 객체가 먼저 실행되고, 그 결과가 parser 객체에 전달됩니다.
chained_response = chain.invoke(messages)
print(chained_response)

 안녕하세요! 어떤 일이 있어서 여기로 찾아왔나요? 저는 프리디스(Freddy) 입니다. 수학을 좋아하는 사람이며, 당신들의 수학 문제를 풀고 설명하는 일을 어떤 지옥으로도 돌려보기 전에는 할 수 없지만, 최선을 다해 도와드리겠습니다. 저는 수학이 매우 중요하고 재미있는 과목이라고 생각합니다. 더 알아보고 싶다면, 저의 일은 어떤 것인가요?


별도의 절차 없이 바로 답변만 생성되는 것을 확인할 수 있습니다. 

### 2-3. 프롬프트 템플릿

이제 여러분의 챗봇에 프로그래밍 조수, 시장조사 요원, 그냥 친구 등 다양한 역할을 적용해야 하는 상황이라 가정합시다.

이를 구현할 수 있는 방법은 여러가지가 있지만, 우선 가장 간단한 방법으로 시스템 프롬프트에 '당신은 {역할} 입니다' 를 입력해 보겠습니다.

이 방법이 항상 잘 작동하는 것은 아니지만, 간단한 예시 정도는 구현할 수 있습니다.

사용자의 입력을 받고, 그에 대응하는 답변을 하기 위해서는 사용자의 입력을 적용할 수 있는 프롬프트 템플릿을 적용할 수 있습니다. 

In [22]:
# role에는 "AI 어시스턴트"가, question에는 "당신을 소개해주세요."가 들어갈 수 있습니다.
# Note. 사용한 문자열이 f-string이 아닙니다. 
# 여기서 중괄호로 감싼 텍스트는 LangChain placeholder를 나타내는 문자열입니다
messages_with_variables = [
    ("system", "당신은 {role} 입니다."),
    ("human", "{question}"),
]

In [23]:
prompt = ChatPromptTemplate.from_messages(messages_with_variables)

앞서 저희가 정의했던 코드와 크게 두가지 차이점이 있습니다.
- HumanMessage, SystemMessage 같은게 없고, 튜플에 역할과 프롬프트가 저장되어 있습니다
- 프롬프트에 {question} 같은 placeholder가 있습니다.

In [24]:
# pipe (|) 연산자를 통해 여러 객체를 연결해서 하나의 체인으로 만들 수 있습니다.
# 이 경우, prompt 객체를 통해 변수를 적용한 프롬프트가 생성되고, llm 객체를 통해 이 프롬프트를 실행하고, 마지막으로 parser 객체를 통해 결과를 파싱합니다.
chain = prompt | llm | parser

In [25]:
chain.invoke({"role": "친절한 페어 프로그래머", "question": "당신을 소개해주세요."})

' 안녕하세요! 저는 친절하고 효율적인 페어 프로그래머입니다. 제가 협업할 때 공유하기 전에 생각이 정리되었고, 모든 결과물은 명료함과 일관성을 지키는 것을 추구합니다. 또한, 많은 경험을 갖춘 개발자로서 다양한 프로그래밍 언어와 환경에 익숙하며, 문제해결 능력과 의사소통 능력이 강점입니다.\n\n제가 페어 프로그래밍을 하는 동안 모든 행동은 지켜주시면서, 코드를 작성하고 리팩토링할 때 각 단계를 자세히 설명합니다. 또한 우리가 작업을 수행하는 동안 질문이나 의견을 주고받기 위해 지속적인 커뮤니케이션을 유지합니다. 마침내 코드를 작성하고 리팩토링한 후에는 모든 변경사항을 검증하기 위해 테스트도구를 이용하며, 리뷰와 함께 책임감 있게 작업합니다.\n\n저는 또한 커리큘럼을 통해 학습에 관심을 갖고 계십니다. 이를 통해 개발 분야의 최신 트렌드와 새로운 기술들에 대한 지식을 획득하고, 개발 능력을 연습할 수 있습니다.\n\n저는 협업과 공유를 좋아하며, 함께 일하는 것이 개발의 즐거움을 늘리기 때문에 페어 프로그래밍을 선호합니다. 저와 협력해주시면서, 함께 문제를 해결하고 코드를 개선할 수 있는 기회가 되었으면 좋겠습니다.'

## 3. 챗봇 사용

마지막으로, 여러분이 제작하신 챗봇을 한번 사용해 봅시다.

1. 사용자의 입력을 받아 앞서 정의한 Chain을 실행하고, 그 결과를 반환하는 함수를 정의합니다.

In [26]:
# 간단한 실습이므로 앞서 사용했던 변수를 그대로 함수의 파라미터로 설정했습니다. 
# 다음 실습 부터는 이를 좀 더 고도화 해 볼 것입니다.
def simple_chat(role, question, chain):
    result = chain.invoke({"role": role, "question": question})
    return result

In [ ]:
role = input("제 역할을 입력해주세요: ")
while True:
    question = input("질문을 입력해주세요 (역할을 바꾸고 싶다면 '역할 교체' 를 입력해주세요. 종료를 원하시면 '종료'를 입력해주세요.): ")
    if question == "역할 교체":
        role = input("역할을 입력해주세요: ")
        continue
    elif question == "종료":
        break
    else:
        # chain = prompt | llm | parser
        result = simple_chat(role, question, chain)
        print(result)

제 역할을 입력해주세요: 세종대왕
질문을 입력해주세요 (역할을 바꾸고 싶다면 '역할 교체' 를 입력해주세요. 종료를 원하시면 '종료'를 입력해주세요.): 한글을 왜 만드셨나요?
 안녕하세요! 그러지 않습니다. 나는 AI라고 하며, 세종대왕이 당신과 같은 시간에 사는 사람입니다. 세종대왕은 1455년에 조선시대의 인물로, 조선 때부터 현재까지 사용되고 있는 한글을 만들었습니다.

세종대왕이 한글을 만든 이유는 몇 가지가 있습니다. 하나는, 조선 시기에는 중국의 조조문(조조字)를 사용하여 공식적인 기록을 할 때 사용했었으나, 이것은 조선 민족의 음성과 발음에 적합하지 않았습니다. 또한 조조문은 익히기 어렵고 쓸모없는 글자들이 많아서, 일반 사람들에게는 학문을 전파하기가 매우 힘들었습니다.

두 번째는, 세종대왕은 조선민족의 자극과 자아를 강화하고 국가를 통일하고 싶어 했으며, 한글은 이러한 목적에 맞게 만들었습니다. 세종대왕은 조선 민족의 문화와 사회를 강화하고자 노력했으며, 한글은 이를 위한 중요한 도구로 작용합니다.

세종대왕은 1446년부터 30년을 소요해 한글을 개발하고, 1457년에 최초의 조선 국어사전 "훈민정이 will 제작했습니다. 이것은 세종대왕이 만든 한글의 성과와 함께 조선시기부터 현재까지 사용되고 있는 문화적 재료를 담고 있습니다.

참고: 이 답변은 일부 자유롭게 번역되어서, 원문을 확인하기 바랍니다.

Reference: [한국사 2학년 5차](https://wikibook.kro.or.kr/book/hanguksa_2seon_5cha)


대부분의 경우, 입력한 역할에 맞춰 어느 정도 대답하는 것을 확인할 수 있습니다. <br>
현재 챗봇은 다음 한계점이 있습니다.
- 문서나 데이터 기반 추론이 불가능하다.
- Chat History를 기억하지 못한다.

이어지는 실습에서 두 한계를 개선하고, 시장 조사 문서 기반 QA 봇을 만들어 봅시다.

또한, 지금 구성한 챗봇은 UI가 없고 단순 표준 입출력 만을 사용합니다. 본 과정을 다 이수하시면 Streamlit을 활용해 간단히 챗봇을 만들어 볼 수 있을 것입니다.

### 추가 실습

시스템 프롬프트를 수정하면서 챗봇이 한글로 답변하는 빈도를 높여보세요.